In [1]:
import sys
sys.path.append('../TTI/')
%load_ext autoreload
%autoreload 2

# Taksonomia, identyfikacja tekstu

Dany jest fragment hierarchii klasyfkacji tematycznej z Wikipedii (https://en.wikipedia.org/wiki/Category:Main_topic_classifications) w postaci pliku CSV.
Klasyfkacja jest grafem spójnym, gdzie węzły są tematami, a krawędzie reprezentują uszczegółowienie tematu.

Celem projektu jest zapropnowanie i przetestowanie mechanizmu automatycznej klasyfikacji tekstu Wejściem jest plik tekstowy w języku angielskim. Wyjściem jest zbiór węzłów w/w klasyfikacji tematycznej.


## Dane wejściowe

Dane wejściowe do zadania do graf spójny o 225765 węzłach, kady węzeł reprezentuje jedną kategorię. Graf nie jest uporządkowanym drzewem, może również zawierać pętle.

In [2]:
from TTI.CategoriesGraph import CategoriesGraph

categories = CategoriesGraph()

Reading topics graph


In [3]:
print("Ilość krawędzi", categories._edge_list.shape)
print("Ilość węzłów", categories._graph.number_of_nodes())

Ilość krawędzi (339250, 2)
Ilość węzłów 225765


## Zbiór treningowy

Zbiór treningowy został przygotowany z wykorzystaniem notebooka `01-tti-training-set-generate.ipynb`. Tam jest też więcej informacji o procesie generacji.

In [6]:
from TTI.config import DATABASE_PATH
import sqlite3
import pandas as pd
import json
import numpy as np
from tensorflow.keras.utils import to_categorical

table_name = "training_set_25"
connection = sqlite3.connect(DATABASE_PATH)


In [7]:
dataset = pd.read_sql('select * from {}'.format(table_name), connection)

In [8]:
dataset["Representation"] = dataset["Representation"].apply(lambda i : json.loads(i))
dataset["Category"] = dataset["Category"].apply(lambda i : i[9:])
dataset["Words"] = dataset["Words"].apply(lambda i : json.loads(i))

In [9]:
print("Dataset size:", dataset.shape)
print("Numeric represntation vector size:", len(dataset.iloc[12]["Representation"]))
print("Number of nodes in the graph:", len(dataset.iloc[12]["Words"]))

Dataset size: (225765, 3)
Numeric represntation vector size: 300
Number of nodes in the graph: 25


In [10]:
dataset

,Category,Words,Representation
0,Main_topic_classifications,"[academic, culture, human, entertainment, heal...","[-0.3755445182323456, 0.010519789531826973, -0..."
1,Main topic articles,"[academic, culture, human, entertainment, heal...","[-0.40671899914741516, 0.013835961930453777, -..."
2,Academic disciplines,"[academic, art, academics, euthenics, studies,...","[-0.09239675104618073, -0.46590009331703186, -..."
3,Subfields by academic discipline,"[subfield, academic, areas, evolutionary, fiel...","[0.085173599421978, 0.010392077267169952, -0.3..."
4,Scholars by subfield,"[subfield, academic, architects, studies, clas...","[-0.15292514860630035, -0.5975006222724915, -0..."
...,...,...,...
225760,World Wide Web stubs,"[internet, wide, system, technology, bioinform...","[0.216136172413826, -0.024581177160143852, -0...."
225761,Internet publication stubs,"[service, wide, entertainment, online, news, s...","[0.2748589515686035, 0.2310565859079361, -0.34..."
225762,Website stubs,"[websites, service, wide, entertainment, onlin...","[0.1632257103919983, 0.16291794180870056, -0.2..."
225763,Wikimedia Foundation stubs,"[websites, service, wide, entertainment, onlin...","[0.19932252168655396, 0.19686073064804077, -0...."


## Wyszukiwanie najbardziej podobnych wektorów

Do klasyfikacji posłuże się obliczaniem odległości geometrycznej pomiędzy wektorami reprezentacji doc2vec. Wektory o najmniejszej odległości zostaną zakwalifikowane jako najbardziej podobne.

In [32]:
dataset.loc[dataset['Category'] == "Machine learning algorithms"]


,Category,Words,Representation
2692,Machine learning algorithms,"[checksum, algorithmic, trading, compression, ...","[0.302137166261673, 0.3030090630054474, -1.029..."


In [34]:
from scipy import spatial

name = dataset["Category"][2692]
vector = dataset["Representation"][2692]

print("Name of category", name)

Name of category Machine learning algorithms


Teraz należe znaleźć najbardziej podobne kategorie. 

In [35]:
import tqdm

def find_simmilar(vector, count, df):
    """ Finds 'count' best matching categories with vectors simmilar to 'vector'"""
    categories = []
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0]):
        vec = row["Representation"]
        name = row["Category"]
        categories.append((name, spatial.distance.cosine(vector, vec)))
    sorted_categories = sorted(categories, key=lambda i: i[1])
    return sorted_categories[0:count]


In [40]:
best_matching = find_simmilar(vector, 20, dataset)

100%|██████████| 225765/225765 [00:39<00:00, 5672.32it/s]


In [41]:
best_matching

[('Machine learning algorithms', 0.0),
 ('Heuristic algorithms', 0.03048611901635534),
 ('Cryptographic algorithms', 0.033841232971325574),
 ('Computer arithmetic algorithms', 0.034831473629752474),
 ('Data mining algorithms', 0.03487279219354722),
 ('Compression algorithms', 0.035516547716642144),
 ('Digit-by-digit algorithms', 0.03591962687260464),
 ('Algorithms', 0.03608325172873006),
 ('Bioinformatics algorithms', 0.03669657755224942),
 ('Approximation algorithms', 0.03737963968331992),
 ('Statistical algorithms', 0.037658969638839856),
 ('Quantum algorithms', 0.037759472173684916),
 ('Graph algorithms', 0.038569196633341796),
 ('Pseudo-polynomial time algorithms', 0.039990344459672755),
 ('Routing algorithms', 0.04005739003276043),
 ('Selection algorithms', 0.04025399420522191),
 ('Algorithm description languages', 0.043534683809688834),
 ('Calendar algorithms', 0.045350172497429675),
 ('Distributed algorithms', 0.04660966816597345),
 ('Streaming algorithms', 0.04691549780262483)]

Jak widać na przykładzie powyżej dla kategorii `Machine learning algorithms` algorytm znalazł 20 najbardziej podobnych klas. Najbardziej podobna okazała się kategoria `Heuristic algroithms`.

In [44]:
best_matching[1]

('Heuristic algorithms', 0.03048611901635534)

## Klasyfikacja dokumentu tekstowego

Mając już algorytm będący w stanie porównać 2 wektory reprezentacji numerycznej `doc2vec` można przejść do właściwej implementacji zadania, czyli klasyfikacji prawdziwego artykułu. 

W pierwszym kroku należy przeprowadzić ekstrakcję zbioru słów charakterystycznych dla danego dokumentu tekstowego. Słowa te zostaną następnie wykorzystane do generacji wektora numerycznej reprezentacji artykułu przy użyciu modeulu `doc2vec`. 

In [74]:
from TTI.TextDocument import get_document_representation, get_article_content

# Finds Wikipedia article by name and downloads it using Wikipedia API
article = get_article_content("K-nearest_neighbors_algorithm")

representation = get_document_representation(article, words_count=75) # How many od the words found in the article should when creating vector

document_words = representation['words']
document_vector = representation['vector']

print("Found words: ", document_words)

Found words:  ['class', 'datum', 'example', 'point', 'classification', 'training', 'neighbor', 'algorithm', 'feature', 'distance', 'error', 'classifier', 'rate', 'set', 'prototype', 'label', 'number', 'analysis', 'reduction', 'outlier', 'regression', 'value', 'weight', 'neighbour', 'search', 'dimension', 'data', 'input', 'object', 'distribution', 'vector', 'space', 'query', 'border', 'ratio', 'test', 'problem', 'approach', 'result', 'extraction', 'method', 'accuracy', 'technique', 'step', 'map', 'density', 'boundary', 'decision', 'case', 'output', 'property', 'function', 'scale', 'x', 'sample', 'constant', 'metric', 'variable', 'way', 'representation', 'effect', 'information', 'size', 'risk', 'expansion', 'term', 'order', 'figure', 'fig', 'k', 'vote', 'average', 'type', 'computation', 'evaluation']


In [71]:
print("Numeric vector: ", document_vector)

Numeric vector:  [-0.12346401810646057, 0.9341837167739868, -1.0335981845855713, -1.4343435764312744, 0.8505613207817078, 1.2286378145217896, -0.237470343708992, -0.26575976610183716, -1.8698941469192505, -0.7291629910469055, -0.5930266976356506, 0.4624477028846741, 0.7712889909744263, -0.6161341071128845, -0.445095956325531, -0.295337438583374, -0.14848719537258148, -0.5035552978515625, 0.048526037484407425, 0.6052327156066895, -0.34907904267311096, 0.15448474884033203, -0.1170039027929306, 0.0013519921340048313, 0.42283666133880615, -0.7253648638725281, -0.07505182921886444, 0.012992607429623604, -0.7499564290046692, -0.8154374361038208, -0.8735315799713135, 0.06651856750249863, 0.28670763969421387, 0.6039268970489502, 0.14671115577220917, 0.6317280530929565, 0.33257314562797546, -0.1257544904947281, 1.4722071886062622, 0.03883741796016693, -0.6392332315444946, 0.8024627566337585, -0.04716314375400543, 0.9875960946083069, -0.4332956075668335, 0.8968942761421204, 0.08390925824642181, 

In [75]:
best_matching = find_simmilar(document_vector, 5, dataset)


100%|██████████| 225765/225765 [00:38<00:00, 5926.12it/s]


In [76]:
best_matching

[('Object recognition and categorization', 0.6077248245626808),
 ('Internet advertising methods', 0.6126320066959126),
 ('Contextual advertising', 0.6136035204571095),
 ('Learning in computer vision', 0.6146926757010172),
 ('Air force units and formations by type', 0.6151154501367266)]

Algorytm wykrył 5 kategorii z grafu wejściowego do których najbardziej "pasuje" artykuł o "K nearest neighbors algorithm".

Są to klasy:
* Object recognition and categorization
* Internet advertising methods
* Contextual advertising
* Air force units and formations by type